In [1]:
import os
import sys
home_dir = "../../"
module_path = os.path.abspath(os.path.join(home_dir))
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc

import matplotlib.pyplot as plt
%matplotlib inline 

In [180]:
pathogenicity_type = "pathogenic" # pathogenic, likely_pathogenic

# root_model = "tape_rao"
# model_name = "protbert" # unirep, protbert

# root_model = "sequnet_dunham"
# model_name = "sequnet"

# root_model = "esm_rives"
# model_name = "esm2_t33_650M_UR50D" # esm1b_t33_650M_UR50S, esm1v_t33_650M_UR90S, esm2_t33_650M_UR50D

root_model = "bioembeddings_dallago"
model_name = "prottrans_t5_xl_u50" # plus_rnn, prottrans_bert_bfd, prottrans_albert_bfd, prottrans_xlnet_uniref100, prottrans_t5_bfd, prottrans_t5_uniref50, prottrans_t5_xl_u50

In [84]:
import seaborn as sns
def plot_distritution_boxplot(result_df):
    fg = sns.catplot(data=result_df, x="pred", y="class", order=[pathogenicity_type, "neutral",], kind="box")
    # fg.set_xlabels("Prediction scores")
    

In [85]:
def plot_distribution(result_df):
    pathogenic = result_df[result_df["class"]==pathogenicity_type]
    neutral = result_df[result_df["class"]=="neutral"]

    # pathogenic_pred_normalized=(pathogenic["pred"]-pathogenic["pred"].min())/(pathogenic["pred"].max()-pathogenic["pred"].min())
    # neutral_pred_normalized=(neutral["pred"]-neutral["pred"].min())/(neutral["pred"].max()-neutral["pred"].min())

    bins = 100 # 20, 100, 500
    # plt.hist(pathogenic_pred_normalized, bins=bins, density=True, log=True, alpha=.5, label=f"Pathogenic-{model_name}")
    # plt.hist(neutral_pred_normalized, bins=bins, density=True, log=True, alpha=.5, label=f"Neutral-{model_name}")

    # print(result_df["class"])
    plt.hist(pathogenic["pred"], bins=bins, density=False, log=False, alpha=.6, label=f"{pathogenicity_type}-{model_name}")
    plt.hist(neutral["pred"], bins=bins, density=False, log=False, alpha=.4, label=f"neutral-{model_name}")
    
    plt.legend()
    plt.show()

In [181]:
aucs = []
for analysis_no in range(10):
    result_df = pd.read_csv(home_dir+f"models/{root_model}/outputs/{model_name}/{pathogenicity_type}/{str(analysis_no)}.csv", sep="\t")

    # this has no effect on the metric, but required for plotting the fig in the same x-axis scale.
    result_df["pred"]=(result_df["pred"]-result_df["pred"].min())/(result_df["pred"].max()-result_df["pred"].min())

    result_df.loc[result_df["class"]==pathogenicity_type, "class_numeric"] = 0
    result_df.loc[result_df["class"]=="neutral", "class_numeric"] = 1

    # score = roc_auc_score(result_df["class"], result_df["pred"])
    # if score < 0.5: 
    #     score = 1 - score
    # print(f"{score:.3f}")
    # aucs.append(score)

    precision, recall, _ = precision_recall_curve(result_df["class_numeric"], result_df["pred"])
    score = auc(recall, precision)
    if score < 0.5: 
        score = 1 - score
    print(f"{score:.3f}")
    aucs.append(score)


    # plot_distribution(result_df)
    # plot_distritution_boxplot(result_df)
    
    # break

print(f"{np.mean(aucs):.3f}")

0.509
0.501
0.508
0.504
0.501
0.512
0.505
0.503
0.510
0.503
0.506
